# The $k$-median Zoo

This writeup contains a collection of constrained $k$-median clustering instances that demostrate counterintuitive properties of the $k$-median objective function when we impose the following additional (somewhat non-standard) requirements:
1. Each point is assigned to exactly $p$ clusters. The number $p$ is called the replication factor. (Note that when $p > 1$, the clusters will no longer be disjoint sets).
2. Each cluster must contain at least $\ell$ points. 
3. Each cluster must contain at most $L$ points.

## $k$-median instances and graph notation

To specify a $k$-median instance, we need to give a collection of points $S$, a metric $d : S \times S \to [0,\infty)$, the replication factor $p \in \mathbb{N}$, and the lower and upper bounds on the cluster sizes, $\ell$ and $L$, respectively. For most real-world examples, the set $S$ is a finite set of points in Euclidean space $\mathbb{R}^d$ and the metric is taken to be the $2$-norm distance, $d(x,y) = \Vert x - y \Vert_2$. Most of the examples in the $k$-median zoo, however, are more abstract. We will often take the set of points to be $S = \{1, \dots, n\}$ and encode the problem entirely in the metric $d$.

We will use the following graphical notation to describe the metric function $d$. Each node in the graph corresponds to a subset of the points in $S$, and the nodes taken together form a partition of $S$. Since the identity of the points doesn't matter, we simply label each node with the number of points it contains. If the label is ommitted, then the node contains exactly one point. The distances between points are determined by the edges in the graph as follows: The distance from any point to itself is always zero. If there is an edge between two nodes $a$ and $b$ in the graph, this means that the distance from any point in $a$ to any point in $b$ is $1$, and vice-versa. All other distances are $2$, including the distance between two points that belong to the same node. For example, the following graph describes a metric on $S = \{1,\dots,4\}$ points, with distances given by $d(i,i) = 0$ for each $i$ in $S$, $d(1,3) = d(2,3) = d(3,4) = 1$ and all other distances equal to $2$:
<img src="figures/simple.png"/>

## Surprise: ${\rm OPT}_k$ is not a decreasing function of $k$

For $k$-median clustering with no lower bounds, the optimal objective value, which we denote by ${\rm OPT}$ is a decreasing function of $k$. Surprisingly, when we add lower bounds on the cluster sizes, this is no longer true. Conside the following example with $n+1$ points:
<img src="figures/one_star.png"/>
Suppose the lower bound on the cluster size is $\ell = 3$. The optimal objective value for $k=1$ of $n$ is achieved when we take the point on the left as the center. For $k=2$, the optimal clustering is to have the point on the left as one center, and one of the $n$ points on the right as the second center. 

The code below optimally solves the two problems and shows that the above objective values are indeed optimal in the case $n=10$.

In [1]:
using ClusteringTools

In [2]:
function example_1()
    # Construct the above metric
    graph = DistanceGraph([1,10])
    add_edge!(graph, 1, 2)
    metric = make_metric(graph)

    # Call the exact MIP-solver based k-median clustering algorithm
    #  Order of arguments: metric, k, p, ℓ, L
    sol_k1 = exact_clustering(metric, 1, 1, 3, size(metric), :KMedian)
    sol_k2 = exact_clustering(metric, 2, 1, 3, size(metric), :KMedian)

    # Print out the objective values for k = 1 and k = 2
    println("OPT(k = 1) = $(objective_value(metric, sol_k1, :KMedian))")
    println("OPT(k = 2) = $(objective_value(metric, sol_k2, :KMedian))")
end
example_1()

OPT(k = 1) = 10.0
OPT(k = 2) = 11.0


The above example can easily be generalized to construct an instance where ${\rm OPT}_k > {\rm OPT}_{k+1}$ for $k > 1$ by creating $k$ copies of the graph.

## Surprise: ${\rm OPT}_k$ is not even a bowl-shaped function of $k$

In light of the above example, one might hope that ${\rm OPT}_k$ is a decreasing function of $k$ until the lower bounds on the cluster sizes become active, after which ${\rm OPT}_k$ is an increasing function of $k$. If this were true, then ${\rm OPT}_k$ would first decrease, and then increase, so that the graph resembles a convex or bowl-shaped function (even though it is only defined at integer points). The following example shows that this is not true either!

In addition to the edges shown in the graph below, add an edge from every node in the top row to every node in the bottom row (these edges have been omitted to keep the figure clean).
<img src="figures/not_bowl_shaped.png"/>
For $n \geq 6$ and $\ell = 15n+1$ we have $OPT_4$ bigger than both $OPT_3$ and $OPT_5$. 

In [3]:
function example_2()
    n = 6
    graph = DistanceGraph([ones(Int, 8); n*[11, 3, 1, 11, 3, 1, 1, 13, 1, 1, 3, 11, 1, 3, 11]])
    # Add edges from top 5 notes to the middle layer
    for a in 1:5
        start = (a-1)*3 + 9
        stop = start + 2
        for b in start:stop
            add_edge!(graph, a, b)
        end
    end
    # Add edges from bottom 3 nodes to the middle layer
    for a in 6:8
        start = 8 + (a-5)
        stop = start + 14
        for b in start:3:stop
            add_edge!(graph, a, b)
        end
    end
    # Also add edges from the top row to the bottom row
    for a in 1:5, b in 6:8
        add_edge!(graph, a, b)
    end
    
    metric = make_metric(graph)
    #                         metric, k, p,    ℓ         L
    sol_k3 = exact_clustering(metric, 3, 1, n*15+1, size(metric), :KMedian)
    sol_k4 = exact_clustering(metric, 4, 1, n*15+1, size(metric), :KMedian)
    sol_k5 = exact_clustering(metric, 5, 1, n*15+1, size(metric), :KMedian)

    println("OPT(k = 3) = $(objective_value(metric, sol_k3, :KMedian))")
    println("OPT(k = 4) = $(objective_value(metric, sol_k4, :KMedian))")
    println("OPT(k = 5) = $(objective_value(metric, sol_k5, :KMedian))")
end
example_2()

OPT(k = 3) = 455.0
OPT(k = 4) = 456.0
OPT(k = 5) = 453.0


### A simpler example demonstrating local maxima in $OPT_k$
There is also a simpler counter example that uses fewer centers and fewer points. Again, in addition to the shown edges, also include an edge between each node in the top layer and each node in the bottom layer.
<img src="figures/simpler_not_bowl_shaped.png"/>
In this case, for $m \geq 5$ and $\ell = 4m+1$ we have that $OPT_3$ is greater than both $OPT_2$ and $OPT_4$.

In [4]:
function example_3()
    m = 5
    graph = DistanceGraph([ones(Int, 6) ; m*[1,3,1,3,1,3,1,3]])
    for (a,b) in [(1,7),  (1,8),
                  (2,9),  (2,10),
                  (3,11), (3,12),
                  (4,13), (4,14),
                  (5,7),  (5,9),  (5,11), (5,13),
                  (6,8),  (6,10), (6,12), (6,14)]
        add_edge!(graph, a, b)
    end
    # Also add an edges between top row and bottom row
    for a in 1:4, b in 5:6
        add_edge!(graph, a, b)
    end
    
    metric = make_metric(graph)
    
    #                         metric, k, p,   ℓ         L
    sol_k2 = exact_clustering(metric, 2, 1, m*4+1, size(metric), :KMedian)
    sol_k3 = exact_clustering(metric, 3, 1, m*4+1, size(metric), :KMedian)
    sol_k4 = exact_clustering(metric, 4, 1, m*4+1, size(metric), :KMedian)

    println("OPT(k = 2) = $(objective_value(metric, sol_k2, :KMedian))")
    println("OPT(k = 3) = $(objective_value(metric, sol_k3, :KMedian))")
    println("OPT(k = 4) = $(objective_value(metric, sol_k4, :KMedian))")
end
example_3()

OPT(k = 2) = 84.0
OPT(k = 3) = 85.0
OPT(k = 4) = 82.0
